In [6]:
import requests
import pandas as pd

In [7]:
# Substitua "{API key}" pela sua chave de API real
api_key = "550d57361bde0d30b0dd389950c781ad"

# Latitude e longitude fictícias
latitude = 33.44
longitude = -94.04

# Parâmetros da chamada de API
params = {
    "lat": latitude,
    "lon": longitude,
    "exclude": "hourly,daily",
    "appid": api_key
}

# URL da API
url = "https://api.openweathermap.org/data/3.0/onecall"

# Fazendo a chamada da API
response = requests.get(url, params=params)

# Verificando se a chamada foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    # Aqui você pode manipular os dados da resposta conforme necessário
    print(data)
else:
    print("A chamada da API falhou. Código de status:", response.status_code)


A chamada da API falhou. Código de status: 401


In [8]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [9]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [10]:
requisicao = requests.get(link2)

In [11]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 284.93,
  'feels_like': 284.46,
  'temp_min': 284.51,
  'temp_max': 286.34,
  'pressure': 1023,
  'humidity': 88,
  'sea_level': 1023,
  'grnd_level': 1019},
 'visibility': 10000,
 'wind': {'speed': 1.23, 'deg': 66, 'gust': 1.94},
 'clouds': {'all': 3},
 'dt': 1697260227,
 'sys': {'type': 2,
  'id': 2001569,
  'country': 'BR',
  'sunrise': 1697273279,
  'sunset': 1697319171},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [12]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 284.93,
  'feels_like': 284.46,
  'temp_min': 284.51,
  'temp_max': 286.34,
  'pressure': 1023,
  'humidity': 88,
  'sea_level': 1023,
  'grnd_level': 1019},
 'visibility': 10000,
 'wind': {'speed': 1.23, 'deg': 66, 'gust': 1.94},
 'clouds': {'all': 3},
 'dt': 1697260227,
 'sys': {'type': 2,
  'id': 2001569,
  'country': 'BR',
  'sunrise': 1697273279,
  'sunset': 1697319171},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [22]:
df = requisicao.json()
df = pd.json_normalize(df)
df = pd.DataFrame(df)

In [23]:
df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1697260227,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1019,1.23,66,1.94,3,2,2001569,BR,1697273279,1697319171


In [ ]:
#Salvando na no S3